In [1]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [2]:
dataset = pd.read_csv('../Datasets/Hepatitis/hepatitis_csv.csv')

In [3]:
dataset.drop('protime', axis=1, inplace=True)

In [4]:
dataset.isna().sum()/len(dataset)

age                0.000000
sex                0.000000
steroid            0.006452
antivirals         0.000000
fatigue            0.006452
malaise            0.006452
anorexia           0.006452
liver_big          0.064516
liver_firm         0.070968
spleen_palpable    0.032258
spiders            0.032258
ascites            0.032258
varices            0.032258
bilirubin          0.038710
alk_phosphate      0.187097
sgot               0.025806
albumin            0.103226
histology          0.000000
class              0.000000
dtype: float64

In [5]:
continuous_features = ['age', 'bilirubin', 'alk_phosphate', 'sgot', 'albumin']
for column in continuous_features:
    dataset[column] = dataset[column].fillna(dataset[column].mean())
for column in dataset.columns.drop(continuous_features):
    dataset[column] = dataset[column].fillna(dataset[column].mode().sample(1, random_state=1).values[0])
for column in dataset.select_dtypes('bool'):
    dataset[column] = dataset[column].astype(np.int)
dataset['sex'] = dataset['sex'].replace({'female': 0,'male': 1})

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys


In [6]:
dataset['class'] = dataset['class'].replace({'live': 0,'die': 1})

In [7]:
X, y = dataset.drop('class', axis=1), dataset['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# AUTOML

In [19]:
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train)
y_hat = automl.predict(X_test)

In [20]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.7692307692307693

In [21]:
import pickle
with open('./models/hepatitis_automl.pkl', 'wb') as f:
    pickle.dump(automl, f)

In [8]:
import pickle
with open('./models/hepatitis_automl.pkl', 'rb') as f:
    automl = pickle.load(f)

len(automl.show_models())

27

# RANDOM FOREST

In [8]:
import sklearn.ensemble

model = sklearn.ensemble.RandomForestClassifier(n_estimators=512, random_state=42)
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

In [9]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.8717948717948718

In [10]:
import pickle
with open('./models/hepatitis_rf2.pkl', 'wb') as f:
    pickle.dump(model, f)

In [9]:
import sklearn.ensemble
import pickle
with open('./models/hepatitis_rf2.pkl', 'rb') as f:
    model = pickle.load(f)

In [10]:
y_hat = model.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_hat)

0.8717948717948718

# CIU

In [11]:
from ciu import determine_ciu
import tqdm

In [12]:
feat_list = X_train.columns.tolist()

In [13]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = determine_ciu(X_test.iloc[i:i+1], model.predict_proba, X_train.to_dict('list'), samples = 1000, prediction_index = 1)
        exp_list = [[feat_list.index(i), exp.ci[i]] for i in exp.ci]
        exp1.append(exp_list)
    return np.array(exp1)

In [14]:
exp1 = exp_fn_blk(X_test)
exp2 = exp_fn_blk(X_test)

100%|██████████| 39/39 [00:54<00:00,  1.40s/it]


In [15]:
np.save('./explanations/hepatitis_ciu1.npy', exp1)
np.save('./explanations/hepatitis_ciu2.npy', exp2)

In [29]:
import metrics

In [31]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [32]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(feat_list))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [33]:
i, s, sb

((17.94871794871795, 32, 39), (0, 39, 1521, 0.0), (14, 39))

In [34]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [35]:
sim

0.6829878898876597

In [36]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(X_test))):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(2, dtype=int)
    np.put(y, y_test.iloc[i], 1)
    example = metrics.FeatureAttribution(model, X_test.to_numpy()[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

100%|██████████| 39/39 [01:17<00:00,  1.99s/it]


In [37]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

0.20520679866470337
0.48717948717948717
8.23076923076923
0.21164522230736263
0.0
0.0


In [38]:
metrics.calc_trust_score(model, X_test.to_numpy(), exp1, 3, X_train.columns.to_list())

100%|██████████| 39/39 [03:28<00:00,  5.36s/it]


0.3034188034188034

# RULEFIT

In [8]:
from skrules import SkopeRules
import metrics_rules
import time

In [9]:
clf = SkopeRules(max_depth_duplication=2,
                    n_estimators=512,
                    precision_min=0.3,
                    recall_min=0.1,
                    feature_names=X_train.columns.tolist())

In [10]:
start_time = time.time()
clf.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 11.780784368515015 seconds ---


In [11]:
start_time = time.time()
top_rules1 = clf.score_top_rules(X_test)
top_rules2 = clf.score_top_rules(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.07526111602783203 seconds ---


In [12]:
i = metrics_rules.calc_identity_rules(top_rules1, top_rules2)
print(i)

s = metrics_rules.calc_separability_rules(top_rules1)
print(s)

enc_rules = metrics_rules.exp_enc(clf, top_rules1)
sb = metrics_rules.calc_stability_rules(enc_rules, y_test)
print(sb)

(0.0, 39, 39)
(332, 39, 1521, 21.82774490466798)
(17, 39)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [65]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(exp1, X_test_norm)
print(sim)

0.9688432163003059


# RULEMATRIX

In [87]:
continuous_features = ['age', 'bilirubin', 'alk_phosphate', 'sgot', 'albumin']
categorical_features = X_train.columns.drop(continuous_features).tolist()

In [88]:
import rulematrix
import time
import metrics_rules

In [89]:
is_continuous = [True if i in continuous_features else False for i in X_train.columns.tolist()]
is_categorical = [True if i in categorical_features else False for i in X_train.columns.tolist()]

In [90]:
surrogate = rulematrix.surrogate.rule_surrogate(
    model.predict,
    X_train,
    sampling_rate=4,
    is_continuous=is_continuous,
    is_categorical=is_categorical,
    seed=42
)

In [91]:
test_x = X_test.to_numpy()

In [92]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in range(len(xtest)):
        queried_rules = np.arange(surrogate.student.n_rules)[surrogate.student.decision_path(test_x[i].reshape(1,-1)).reshape(-1)]
        exp1.append(queried_rules[-1])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x))

In [93]:
start_time = time.time()
exp1 = exp_fn_blk(test_x)
exp2 = exp_fn_blk(test_x)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.01602959632873535 seconds ---


In [94]:
def enc_exp(exp, n_features):
    enc = []
    for i in range(exp.shape[0]):
        new = np.zeros(n_features)
        for j in surrogate.student.rule_list[exp[i]].clauses:
            new[j.feature_idx] = 1
        enc.append(new)
    return np.array(enc)

In [95]:
enc_exp = enc_exp(exp1, X_train.shape[1])

In [96]:
i = metrics_rules.calc_identity_rules(exp1, exp2)
print(i)

s = metrics_rules.calc_separability_rules(exp1)
print(s)

sb = metrics_rules.calc_stability_rules(enc_exp, y_test)
print(sb)

(0.0, 39, 39)
(440, 39, 1521, 28.928336620644313)
(15, 39)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [97]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(enc_exp, X_test_norm)

In [98]:
sim

1.1212315681063432